In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image data generators
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Set up generators
train_generator = train_datagen.flow_from_directory(
    'Dataset/Train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    'Dataset/Validation',  # Corrected path
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    'Dataset/Test',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)


Found 1322 images belonging to 3 classes.
Found 60 images belonging to 3 classes.
Found 150 images belonging to 3 classes.


In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
import datetime

# Define image data generators
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Set up generators
train_generator = train_datagen.flow_from_directory(
    'Dataset/Train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'  # One-hot encoded labels
)

valid_generator = valid_datagen.flow_from_directory(
    'Dataset/Validation',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'  # One-hot encoded labels
)

test_generator = test_datagen.flow_from_directory(
    'Dataset/Test',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'  # One-hot encoded labels
)

# Define image dimensions
image_height = 150
image_width = 150

# Define the model architecture
model = Sequential([
    Input(shape=(image_height, image_width, 3)),  # Input layer with the shape of the input images
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # 3 classes for 'rust', 'powdery', 'healthy'
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Use categorical_crossentropy for one-hot encoded labels
              metrics=['accuracy'])

# Define the callbacks
checkpoint = ModelCheckpoint('model_best.keras', save_best_only=True, monitor='val_loss')
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# TensorBoard callback to log training progress
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Print model summary
model.summary()

# Train the model with progress displayed and tracked by TensorBoard
history = model.fit(
    train_generator,
    epochs=10,  # Adjust epochs as needed
    validation_data=valid_generator,
    callbacks=[checkpoint, early_stopping, tensorboard_callback],  # Add TensorBoard callback
    verbose=1  # Set verbose=1 to see progress in Jupyter
)


Found 1322 images belonging to 3 classes.
Found 60 images belonging to 3 classes.
Found 150 images belonging to 3 classes.


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,828,739 (18.42 MB)

 Trainable params: 4,828,739 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 252s 5s/step - accuracy: 0.4338 - loss: 1.2556 - val_accuracy: 0.6500 - val_loss: 0.8428
Epoch 2/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 264s 5s/step - accuracy: 0.6869 - loss: 0.7119 - val_accuracy: 0.7000 - val_loss: 0.6655
Epoch 3/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 173s 3s/step - accuracy: 0.7906 - loss: 0.5388 - val_accuracy: 0.7667 - val_loss: 0.4809
Epoch 4/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 187s 4s/step - accuracy: 0.8777 - loss: 0.3250 - val_accuracy: 0.7667 - val_loss: 0.5314
Epoch 5/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 246s 5s/step - accuracy: 0.8992 - loss: 0.3032 - val_accuracy: 0.7833 - val_loss: 0.5231
Epoch 6/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 250s 5s/step - accuracy: 0.9238 - loss: 0.2225 - val_accuracy: 0.8833 - val_loss: 0.3458
Epoch 7/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 220s 5s/step - accuracy: 0.9413 - loss: 0.1996 - val_accuracy: 0.8667 - val_loss: 0.3479
Epoch 8/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 266s 4s/step - accuracy: 0.9543 - loss: 0.1359 - val_accuracy: 0.9000 - v

In [17]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}, Test Loss: {test_loss}")


5/5 ━━━━━━━━━━━━━━━━━━━━ 17s 3s/step - accuracy: 0.9060 - loss: 0.4048
Test Accuracy: 0.9133333563804626, Test Loss: 0.35009583830833435


In [18]:
print("Class indices:", train_generator.class_indices)
print("Class labels:", list(train_generator.class_indices.keys()))



Class indices: {'Healthy': 0, 'Powdery': 1, 'Rust': 2}
Class labels: ['Healthy', 'Powdery', 'Rust']


In [19]:
import os

train_dir = 'dataset/train'
print("Subdirectories in train directory:")
print(os.listdir(train_dir))


Subdirectories in train directory:
['Healthy', 'Powdery', 'Rust']


In [21]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_crop_disease(img_path):
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Rescale to [0, 1]

    prediction = model.predict(img_array)
    class_idx = np.argmax(prediction, axis=1)[0]

    class_labels = list(train_generator.class_indices.keys())
    
    if class_idx >= len(class_labels):
        raise IndexError("Predicted class index is out of range for available class labels.")
    
    class_label = class_labels[class_idx]
    return class_label

# Example usage
disease_prediction = predict_crop_disease('newimg/download.jpg')
print(f'Predicted disease: {disease_prediction}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Predicted disease: Healthy


In [23]:

model.save('model_best.keras')
